In [5]:
library(dplyr)
library(stringr)
library(readr)
library(tidyr)
library(lfe)
library(scales)
library(repr)
library(stargazer)
library(gridExtra)
options(repr.plot.width=10, repr.plot.height=10)
source('~/covid_mobility/paper_code/utils/adjacency_and_cluster_robust_inference.R')
source('~/covid_mobility/paper_code/utils/residualizer+genAlters.R')
source('../utils/labels_and_colors.r')
load('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/social_adj_matrix.RData')
load('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/data_v2.RData')
load('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/conley_selector_matrix.Rdata')

In [2]:
conley.selector.matrix

,1001,1003,1005,1007,1009,1011,1013,1015,1017,1019,⋯,56013,56015,56021,56023,56025,56029,56033,56037,56039,56041
1001,1,1,1,1,1,1,1,1,1,1,⋯,2.542773e-05,6.877848e-06,3.728662e-04,2.985054e-05,6.188003e-05,2.214430e-05,2.894149e-05,2.658651e-05,3.768613e-05,1.216162e-05
1003,1,1,1,1,1,1,1,1,1,1,⋯,4.881919e-05,2.420658e-05,1.247940e-04,1.919712e-05,8.932390e-05,6.377706e-05,3.017285e-05,3.661513e-05,6.892416e-05,2.279094e-05
1005,1,1,1,1,1,1,1,1,1,1,⋯,1.503575e-05,6.572616e-05,5.101400e-05,5.732409e-06,2.096182e-05,1.889264e-05,6.367600e-06,8.799955e-06,8.850604e-06,1.245832e-05
1007,1,1,1,1,1,1,1,1,1,1,⋯,1.694849e-05,7.474688e-06,6.737088e-05,8.368933e-06,3.876538e-05,2.827390e-05,3.890532e-05,2.454678e-05,9.735418e-06,8.779884e-06
1009,1,1,1,1,1,1,1,1,1,1,⋯,2.595817e-05,2.028731e-05,1.240203e-04,1.932705e-05,9.449083e-05,4.544160e-05,4.342157e-05,3.762450e-05,1.451556e-05,1.519681e-05
1011,1,1,1,1,1,1,1,1,1,1,⋯,2.713271e-06,2.221382e-06,3.627325e-05,1.934953e-06,1.297210e-05,6.013247e-06,5.174913e-06,5.599665e-06,6.356359e-06,6.494735e-06
1013,1,1,1,1,1,1,1,1,1,1,⋯,4.413518e-05,3.936286e-06,3.947635e-05,6.006010e-06,3.938737e-05,4.375043e-05,2.017127e-05,2.057050e-05,1.279333e-05,5.663130e-06
1015,1,1,1,1,1,1,1,1,1,1,⋯,2.683448e-05,3.477735e-05,1.292058e-04,2.660684e-05,5.279948e-05,4.652367e-05,1.937664e-05,2.435841e-05,2.764483e-04,1.003326e-05
1017,1,1,1,1,1,1,1,1,1,1,⋯,1.279209e-05,5.131970e-05,5.097371e-05,2.905704e-06,3.206503e-05,1.240165e-05,8.233462e-06,1.055181e-05,3.487420e-04,8.687382e-06
1019,1,1,1,1,1,1,1,1,1,1,⋯,9.807439e-05,6.292299e-05,1.210263e-04,9.493043e-05,4.993965e-05,4.044810e-05,2.283218e-05,9.646980e-05,1.986735e-05,9.419436e-06


In [36]:
######## Main DVs
##### RELATIVE 
# DV           CORRESPONDING PEER VARAIBLE
# ash_rnstu.r  alter_ash_rnstu.r
# fb_btvrc.r   alter_fb_btvrc.r
# ash_nhd.r    alter_ash_nhd.r
# log_mcbgv.r  alter_log_mcgbv.r
##### NON RELATIVE 
# DV           CORRESPONDING PEER VARAIBLE
# fb_rnstu.r   alter_rnstu.r
# sg_nhd.r     alter_nhd.r
# sg_mcbgv.r   alter_mcgbv.r
#####

In [47]:
### IF YOU NEED TO ADD A VARIABLE/INTERACTION
panel %>%
    mutate(ash_rnstu_Xsh = ash_rnstu * stay_home,
           ash_alter_rnstu = asinh(alter_rnstu),
           ash_alter_nhd   = asinh(alter_nhd),
           log_alter_mcbgv = asinh(alter_mcbgv)) %>%
    mutate(alter_rnstu_Xsh = weightedAlters(., WM, ash_rnstu_Xsh),   
           alter_rnstu_Xsh.r = panel_residualizer(alter_rnstu_Xsh),
           ash_alter_rnstu.r = panel_residualizer(ash_alter_rnstu),
           ash_alter_nhd.r   = panel_residualizer(ash_alter_nhd),
           log_alter_mcbgv.r = panel_residualizer(log_alter_mcbgv)) -> panel
### NOTE if VAR is generated in a mutate, then weightedAlters(., WM, VAR) cannot be in the same mutate.

[1]	train-rmse:0.259209+0.000288	test-rmse:0.259296+0.001630 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 3 rounds.

[2]	train-rmse:0.142108+0.000143	test-rmse:0.142375+0.001737 
[3]	train-rmse:0.091606+0.000165	test-rmse:0.092101+0.001429 
[4]	train-rmse:0.073468+0.000143	test-rmse:0.074263+0.000906 
[5]	train-rmse:0.068012+0.000272	test-rmse:0.069055+0.000578 
[6]	train-rmse:0.066524+0.000323	test-rmse:0.067697+0.000361 
[7]	train-rmse:0.066000+0.000447	test-rmse:0.067296+0.000234 
[8]	train-rmse:0.065799+0.000414	test-rmse:0.067301+0.000214 
[9]	train-rmse:0.065600+0.000408	test-rmse:0.067237+0.000145 
[10]	train-rmse:0.065380+0.000413	test-rmse:0.067268+0.000244 
[11]	train-rmse:0.065265+0.000390	test-rmse:0.067256+0.000265 
[12]	train-rmse:0.065195+0.000422	test-rmse:0.067284+0.000290 
Stopping. Best iteration:
[9]	train-rmse:0.065600+0.000408	test-rmse:0.067237+0.000145

[1]	train-rmse:0.250276+0.000055	t

In [10]:
m1 <- felm(fb_rnstu.r ~ 0 + ban_gmr.r + alter_bgmr.r + geo_alter_bgmr.r + stay_home.r + alter_sh.r + geo_alter_sh.r | 
            0 | 
            (alter_rnstu.r ~ 
             iv01 + iv02 + iv03 + iv04 + iv05 + iv06 + iv07 + iv08 + iv09 + iv10 +
             iv11 + iv12 + iv13 + iv14 + iv15 + iv16 + iv17 + iv18 + iv19 + iv20 +
             iv21 + iv22 + iv23 + iv24 + iv25 + iv26 + iv27 + iv28 + iv29 + iv30 +
             iv31 + iv32 + iv33 + iv34 + iv35 + iv36 + iv37 + iv38 + iv39 + iv40 +
             iv41 + iv42 + iv43 + iv44 + iv45 + iv46 + iv47 + iv48 + iv49 + iv50 +
             iv51 + iv52 + iv53 + iv54 + iv55 + iv56 + iv57 + iv58 + iv59 + iv60 +
             iv61 + iv62 + iv63 + iv64 + iv65 + iv66 + iv67 + iv68 + iv69 + iv70 +
             iv71 + iv72 + iv73 + iv74 + iv75 + iv76 + iv77 + iv78 + iv79 + iv80 +
             iv81 + iv82 + iv83 + iv84 + iv85 + iv86 + iv87 + iv88 + iv89 + iv90 +
             iv91 + iv92 + iv93 + iv94) | state_abbv, panel, weights = panel$n, keepCX = T)

sum.m1 <- summarize.felm.fit.with.adj(m1, conley.selector.matrix, panel$state_abbv)
sum.m1$ct

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or indefinite”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”



t test of coefficients:

                       Estimate Std. Error t value  Pr(>|t|)    
ban_gmr.r            -0.0025638  0.0072210 -0.3551    0.7226    
alter_bgmr.r         -0.0030459  0.0171194 -0.1779    0.8588    
geo_alter_bgmr.r      0.0040578  0.0107167  0.3786    0.7050    
stay_home.r          -0.0118442  0.0096800 -1.2236    0.2211    
alter_sh.r            0.0171280  0.0166776  1.0270    0.3044    
geo_alter_sh.r       -0.0023260  0.0094196 -0.2469    0.8050    
`alter_rnstu.r(fit)`  1.7630651  0.3251998  5.4215 5.922e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [11]:
m1 <- felm(fb_rnstu.r ~ 0 + ban_gmr.r + alter_bgmr.r + geo_alter_bgmr.r + stay_home.r + alter_sh.r + geo_alter_sh.r | 
            0 | 
            (alter_rnstu.r ~ 
             iv01 + iv02 + iv03 + iv04 + iv05 + iv06 + iv07 + iv08 + iv09 + iv10 +
             iv11 + iv12 + iv13 + iv14 + iv15 + iv16 + iv17 + iv18 + iv19 + iv20 +
             iv21 + iv22 + iv23 + iv24 + iv25 + iv26 + iv27 + iv28 + iv29 + iv30 +
             iv31 + iv32 + iv33 + iv34 + iv35 + iv36 + iv37 + iv38 + iv39 + iv40 +
             iv41 + iv42 + iv43 + iv44 + iv45 + iv46 + iv47 + iv48 + iv49 + iv50 +
             iv51 + iv52 + iv53 + iv54 + iv55 + iv56 + iv57 + iv58 + iv59 + iv60 +
             iv61 + iv62 + iv63 + iv64 + iv65 + iv66 + iv67 + iv68 + iv69 + iv70 +
             iv71 + iv72 + iv73 + iv74 + iv75 + iv76 + iv77 + iv78 + iv79 + iv80 +
             iv81 + iv82 + iv83 + iv84 + iv85 + iv86 + iv87 + iv88 + iv89 + iv90 +
             iv91 + iv92 + iv93 + iv94) | 0, panel, weights = panel$n, keepCX = T)

sum.m1 <- summarize.felm.fit.with.adj(m1, conley.selector.matrix, panel$state_abbv)
sum.m1$ct

Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”



t test of coefficients:

                        Estimate  Std. Error  t value  Pr(>|t|)    
ban_gmr.r            -2.5638e-03  5.1257e-05  -50.020 < 2.2e-16 ***
alter_bgmr.r         -3.0459e-03  1.4690e-04  -20.734 < 2.2e-16 ***
geo_alter_bgmr.r      4.0578e-03  1.0336e-04   39.259 < 2.2e-16 ***
stay_home.r          -1.1844e-02  7.2167e-05 -164.123 < 2.2e-16 ***
alter_sh.r            1.7128e-02  3.1654e-04   54.110 < 2.2e-16 ***
geo_alter_sh.r       -2.3260e-03  1.7749e-04  -13.105 < 2.2e-16 ***
`alter_rnstu.r(fit)`  1.7631e+00  2.2561e-03  781.481 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
